In [1]:
from heapq import heappush, heappop

from board2 import Board2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import random
from controller import ActionController
from tqdm.auto import tqdm
from torch.distributions import Categorical
from collections import deque
import matplotlib.pyplot as plt
import math
import time

In [2]:
class Node:
    def __init__(self, board, player):
        self.board = board
        self.controller = ActionController(board)
        self.children = []
        self.p = player
        self.qsa = 0
        self.psa = 0
        self.nsa = 0
        self.a = None

    def search(self, player):
        winner = self.get_winner()
        if winner is not None:
            self.nsa += 1
            return winner

        if len(self.children) == 0:

            actions = []
            for a in self.controller.get_available_moves():
                board = self.board.copy()
                p = -self.p
                n = Node(board, p)
                n.a = a
                n.controller.execute_action(a)
                board.step()
                actions.append(a)
                self.children.append(n)

            boards = torch.cat([x.board.grid.clone().unsqueeze(0) for x in self.children], dim=0)

            with torch.no_grad():
                pl, vl = player(boards)

            for i, (n, a) in enumerate(zip(self.children, actions)):

                if n.get_winner() is not None:
                    n.qsa = n.get_winner()
                else:
                    n.qsa = vl[i].item()
                n.psa = pl.exp()[i, a].item()
                n.board.swap_enemy()

            self.qsa = torch.mean(vl).item()
            self.nsa = len(self.children)

            return self.qsa

        best = max(self.children, key=lambda x: x.qsa * x.prob + 1.5 * x.psa * math.sqrt(sum([x.nsa for x in self.children]) / (x.nsa + 1)))

        res = best.search(player)

        self.qsa = (self.qsa * self.nsa + res) / (self.nsa + 1)
        self.nsa += 1

        return res

    def simulate(self, player):
        winner = self.get_winner()
        if winner is not None:
            return winner

        with torch.no_grad():
            pl, vl = player(self.board.grid.clone().unsqueeze(0))
        return vl.item()


    def get_winner(self):

        if self.controller.is_win():
            return 1
        elif self.controller.is_lose():
            return -1
        elif self.controller.is_block():
            return 0
        return None

    def get_improved_policy(self):
        fp = [0 for x in range(self.controller.get_action_space())]
        s = sum([x.nsa for x in self.children])
        for x in self.children:
            fp[x.action] = x.nsa / s
        return fp

    def get_policy(self):
        s = sum([x.nsa for x in self.children])
        return [x.nsa / s for x in self.children]

In [6]:
def pos_encode(max_len, d_model, dtype=torch.float32):
    position = torch.arange(max_len).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
    pe = torch.zeros(max_len, d_model, dtype=dtype)
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    return pe

class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        self.pos = pos_encode(9, 16)

        self.q = nn.Linear(16, 16)
        self.k = nn.Linear(16, 16)
        self.v = nn.Linear(16, 16)
        self.n_heads = 16
        self.n_dim = 16

        self.dropout = nn.Dropout(0.1)
        self.lc = nn.Linear(16, 16)


    def forward(self, x):
        wq = self._head_reshape(self.q(x))
        wk = self._head_reshape(self.k(x))
        wv = self._head_reshape(self.v(x))

        a = wq @ wk.transpose(2, 3) / math.sqrt(self.n_dim)
        a = torch.softmax(a, dim=-1)
        a = self.dropout(a)

        o = a @ wv

        return self.lc(self._head_reshape_back(o))

    def _head_reshape(self, x):
        return x.view(x.shape[0], x.shape[1], self.n_heads, self.n_dim // self.n_heads).transpose(1, 2)

    def _head_reshape_back(self, x):
        return x.transpose(1, 2).contiguous().view(x.shape[0], x.shape[2], self.n_dim)

class Enc(nn.Module):
    def __init__(self):
        super(Enc, self).__init__()
        self.attn = Attention()
        self.ln1 = nn.LayerNorm(16)

        self.seq = nn.Sequential(
            nn.Linear(16, 128),
            nn.ReLU(),
            nn.Linear(128, 16)
        )

        self.ln2 = nn.LayerNorm(16)

    def forward(self, x):
        h = self.attn(x)
        h1 = self.ln1(h + x)
        h2 = self.seq(h1)
        return self.ln2(h1 + h2)

class FrodoPolicy(nn.Module):
    def __init__(self, action_space):
        super(FrodoPolicy, self).__init__()

        self.flatten = nn.Flatten(start_dim=1)
        # self.pos = pos_encode(9, 16)
        #
        # self.enc = nn.ModuleList([Enc() for _ in range(4)])
        #
        # self.pl = nn.Sequential(
        #     nn.Linear(16, action_space),
        #     nn.LogSoftmax(dim=-1)
        # )
        #
        # self.vl = nn.Linear(16, 1)

        self.enc = nn.Linear(144, 32)
        self.pl = nn.Sequential(
            nn.Linear(32, action_space),
            nn.LogSoftmax(dim=-1)
        )

        self.vl = nn.Linear(32, 1)



    def forward(self, x):
        fx = self.flatten(x)
        h = self.enc(fx)
        # px = self.pos.unsqueeze(0) + self.flatten(x)
        #
        # for enc in self.enc:
        #     px = enc(px)
        #
        # cv = px.mean(dim=1)
        #
        # return self.pl(cv), self.vl(cv)
        return self.pl(h), self.vl(h)




board = Board2()
controller = ActionController(board)
#
player = FrodoPolicy(controller.get_action_space())
player.eval()

with torch.no_grad():
    res = player(board.grid.clone().unsqueeze(0))

In [5]:
def apply_rewards(samples, p, winner):
    if p == 1:
        if winner == 1:
            rewards = [1 if i % 2 == 0 else -1 for i in range(len(samples))]
        elif winner == -1:
            rewards = [-1 if i % 2 == 0 else 1 for i in range(len(samples))]
        else:
            rewards = [0 for i in range(len(samples))]
    else:
        if winner == 1:
            rewards = [-1 if i % 2 == 0 else 1 for i in range(len(samples))]
        elif winner == -1:
            rewards = [1 if i % 2 == 0 else -1 for i in range(len(samples))]
        else:
            rewards = [0 for i in range(len(samples))]
    return rewards

def episode(player, moves = 100, mcts_iter = 50):
    board = Board2()
    samples = []
    root = Node(board, -1)

    for i in range(moves):
        for _ in range(mcts_iter):
            root.search(player)

        samples.append((root.board, root.get_improved_policy()))
        c = Categorical(torch.Tensor(root.get_improved_policy()))
        move = c.sample().item()
        for c in root.children:
            if c.action == move:
                root = c
                break
        else:
            raise Exception('Invalid move')

        if root.get_winner() is not None:
            return samples, root.prob, root.get_winner()

    return samples, root.p, 0

def train_nn(player, dataset):
    player.train()
    optimizer = optim.Adam(player.parameters(), lr=0.001)
    crit1 = nn.MSELoss()
    crit2 = nn.CrossEntropyLoss()

    lh = []
    for e in range(10):
        el = []
        for b, p, r in DataLoader(dataset, batch_size=4, shuffle=True):
            optimizer.zero_grad()
            pl, vl = player(b)
            loss = crit1(vl.squeeze(), r) + crit2(pl, p)
            loss.backward()
            el.append(loss.item())
            optimizer.step()
        lh.append(sum(el) / len(el))

    return lh


def battle(player, enemy):

    player.eval()
    enemy.eval()

    board = Board2()
    controller = ActionController(board)

    for _ in range(100):

        with torch.no_grad():
            pl, _ = player(board.grid.clone().unsqueeze(0))
        player_move = pl.exp().argmax().item()
        controller.execute_action(player_move)
        board.step()

        if controller.is_win():
            return 1
        elif controller.is_block():
            return 0

        with torch.no_grad():
            pl, _ = enemy(board.grid.clone().unsqueeze(0))
        enemy_move = pl.exp().argmax().item()
        controller.execute_action(enemy_move)
        board.step()

        if controller.is_lose():
            return -1
        elif controller.is_block():
            return 0

    return 0


def test_battle(player, enemy, n = 10):
    wins, draws, loses = 0, 0, 0
    for _ in range(n):
        res = battle(player, enemy)
        if res == 1:
            wins += 1
        elif res == 0:
            draws += 1
        else:
            loses += 1
    return wins / n, draws / n, loses / n




In [7]:
enemy = FrodoPolicy(ActionController(Board2()).get_action_space())
player = FrodoPolicy(ActionController(Board2()).get_action_space())

for e in tqdm(range(10)):
    print('Iteration:', e)
    data_boards = []
    data_policies = []
    data_rewards = []
    player.eval()
    enemy.eval()
    for _ in range(5):

        samples, p, winner = episode(player)

        data_boards.extend([x[0].grid.clone().unsqueeze(0) for x in samples])
        data_policies.extend([x[1] for x in samples])
        data_rewards.extend(apply_rewards(samples, p, winner))

    boards = torch.cat(data_boards, dim=0)
    policies = torch.Tensor(data_policies)
    rewards = torch.Tensor(data_rewards)


    dataset = TensorDataset(boards, policies, rewards)

    lh = train_nn(player, dataset)
    print('Loss:', lh[-1])

    wins, draws, loses = test_battle(player, enemy, 10)
    print('Wins:', wins, 'Draws:', draws, 'Loses:', loses)
    if wins > 0.55:
        print('Enemy updated.')
        enemy.load_state_dict(player.state_dict())



  0%|          | 0/10 [00:00<?, ?it/s]

Iteration: 0


/home/guilherme/PycharmProjects/todd/venv/lib/python3.12/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/guilherme/PycharmProjects/todd/venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Loss: 3.238573760402446
Wins: 0.0 Draws: 1.0 Loses: 0.0
Iteration: 1
Loss: 3.242127052942912
Wins: 0.1 Draws: 0.9 Loses: 0.0
Iteration: 2
Loss: 3.172249781001698
Wins: 0.1 Draws: 0.9 Loses: 0.0
Iteration: 3
Loss: 3.089030446232976
Wins: 0.0 Draws: 1.0 Loses: 0.0
Iteration: 4
Loss: 3.0533790473015077
Wins: 0.1 Draws: 0.9 Loses: 0.0
Iteration: 5
Loss: 2.971366214752197
Wins: 0.0 Draws: 1.0 Loses: 0.0
Iteration: 6
Loss: 3.09035378236037
Wins: 0.1 Draws: 0.9 Loses: 0.0
Iteration: 7
Loss: 2.930754208564758
Wins: 0.0 Draws: 1.0 Loses: 0.0
Iteration: 8
Loss: 2.976498377950568
Wins: 0.0 Draws: 1.0 Loses: 0.0
Iteration: 9
Loss: 3.093575565253987
Wins: 0.0 Draws: 1.0 Loses: 0.0


In [14]:

start = time.time()

with torch.multiprocessing.Pool(4) as pool:
    res = pool.map(episode, [player for _ in range(5)])

end = time.time()

print('Time:', end - start)

KeyboardInterrupt: 